In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import gc
import re
from qp_utils import (
    expand_vars, load_year, save_compact_meta,
    decode_labels, show_value_table_json, peek_sav_columns,
)

RAW_DIR = Path("Spss")
BASE_YEAR = 2010
years = list(range(2010, 2023+1))
files = {}
for y in years:
    p = RAW_DIR / f"QP_Empresas_{y}.sav"
    if p.exists():
        files[y] = p

print()
def strip_year_tag_any(name: str) -> str:
    s = re.sub(r'_(\d{2})(?=(_|$))', '', name)
    s = re.sub(r'(?<=\D)(\d{2})$', '', s)
    return s
def base_vars_from_sav(path: Path) -> set[str]:
    cols = peek_sav_columns(path)["var"].tolist()
    return {strip_year_tag_any(v) for v in cols}

vars_by_year: dict[int, set[str]] = {}
for y, p in sorted(files.items()):
    s = base_vars_from_sav(p)
    vars_by_year[y] = s
    print(f"Year {y}: {len(s)} base variables in {p.name}")

def base_vars_from_sav(path: Path) -> set[str]:
    cols = peek_sav_columns(path)["var"].tolist()
    return {strip_year_tag_any(v) for v in cols}

def build_presence_table(files: dict[int, Path], base_year: int, diff_only: bool = True):
    if base_year not in files:
        raise ValueError(f"base_year {base_year} not in files")

    # Build sets of base variable names per year
    vars_by_year: dict[int, set[str]] = {y: base_vars_from_sav(p) for y, p in files.items()}

    # Union of all variable base-names, so we see extras not in base too
    all_vars = sorted(set().union(*vars_by_year.values()))

    # Build presence matrix
    years_sorted = [base_year] + sorted([y for y in files if y != base_year])
    data = {
        y: [v in vars_by_year[y] for v in all_vars]
        for y in years_sorted
    }
    presence = pd.DataFrame(data, index=all_vars)

    # Optionally filter to differences vs base column
    base_col = years_sorted[0]
    if diff_only:
        mask = (presence.T != presence[base_col]).any()
        presence = presence.loc[mask]

    return presence

presence_df = build_presence_table(files, BASE_YEAR, diff_only=True)

# See the raw boolean table (if you want)
presence_df.head(40)

# wanted_tpl = ["ano_{yy}","nuemp_{yy}","nuest_{yy}","ntrab_{yy}","sexo_{yy}","idade_{yy}_TB_COD","dtadm_{yy}"]

# df_2008, meta_2008 = load_year(2008, files[2008], cols=expand_vars(wanted_tpl, 2008))
# df_2009, meta_2009 = load_year(2009, files[2009], cols=expand_vars(wanted_tpl, 2009))

# save_compact_meta(meta_2008, 2008, df_2008)
# save_compact_meta(meta_2009, 2009, df_2009)

# df = pd.concat([df_2008, df_2009], ignore_index=True, copy=False)
# del df_2008, df_2009, meta_2008, meta_2009; gc.collect()

# # decode labels where available (e.g., sexo)
# decode_labels(df, "sexo")

# # quick table for 2009
# show_value_table_json(df, "sexo", 2009, sort_index=True)

In [ ]:
presence_df[[2009, 2008]].head(40)

In [ ]:
peek_sav_columns(files[2009])

In [ ]:
peek_sav_columns(files[2010])